Capstone Week 3 assignment 

Import All Libraries and get html/table content from wikipedia

In [98]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

#get html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table',limit=1)


Parse through Tr then td for each data extracting them into separate list for postal code, borough and neighborhood
Then create a data frame with these list

In [114]:
#define header

rows=[]; cells=[];PostalCode=[];Borough=[];Neighborhood=[]

for t in table:
    rows = t.find_all('tr')

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        postal = cells[0]
        PostalCode.append((postal.text.strip()))

        bor = cells[1]
        Borough.append(bor.text.strip())

        neigh = cells[2]
        Neighborhood.append(neigh.text.strip())


df = pd.DataFrame(
    {'PostalCode': PostalCode,
     'Borough': Borough,
     'Neighborhood': Neighborhood
    })

df
    

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Data clean up by removing Not Assigned values 

In [120]:
df = df[~df['Borough'].isin(['Not assigned'])]
notAssignedList=df.index[df['Neighborhood'] == 'Not assigned'].tolist()

for i in notAssignedList:
    df['Neighborhood'][i].replace('Not assigned',df['Borough'][i])

df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Importing csv and adding lat/long
while running localy update the path to csv file variable path_csv

In [123]:
path_csv = "C:\SRC\capstoneweek3_1b\Geospatial_Coordinates.csv"
data = pd.read_csv(path_csv) 

merged_df = pd.merge(left=df, right=data, how='left', left_on='PostalCode', right_on='Postal Code').drop(columns=['Postal Code'])
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
